<a href="https://colab.research.google.com/github/jkranyak/background_removed_image_tones_to_music_tone_UI/blob/main/Sound_of_your_photos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio numpy pandas Pillow scikit-learn scipy rembg

In [2]:
pip install tensorflow mido pretty_midi

In [3]:
# import tensorflow as tf
# import numpy as np
# import mido
# import pretty_midi

# class NeuralMusicGenerator:
#     def __init__(self, model_path):
#         self.model = tf.keras.models.load_model(model_path)
#         self.sequence_length = 100
#         self.num_notes = 128

#     def generate_melody(self, seed_notes, length):
#         melody = seed_notes.copy()
#         for _ in range(length - len(seed_notes)):
#             sequence = melody[-self.sequence_length:]
#             sequence = np.array(sequence) / self.num_notes
#             sequence = np.reshape(sequence, (1, self.sequence_length, 1))
#             prediction = self.model.predict(sequence)
#             next_note = np.argmax(prediction)
#             melody.append(next_note)
#         return melody

#     def create_midi(self, melody, output_file):
#         midi = pretty_midi.PrettyMIDI()
#         instrument = pretty_midi.Instrument(program=0)  # Piano

#         for i, pitch in enumerate(melody):
#             note = pretty_midi.Note(
#                 velocity=100,
#                 pitch=int(pitch),
#                 start=i * 0.25,
#                 end=(i + 1) * 0.25
#             )
#             instrument.notes.append(note)

#         midi.instruments.append(instrument)
#         midi.write(output_file)

# train or obtain a pre-trained model for this
# generator = NeuralMusicGenerator('path_to_your_model.h5')
# melody = generator.generate_melody([60, 62, 64, 65, 67], 100)
# generator.create_midi(melody, 'generated_melody.mid')

In [4]:
!pip install music21

In [5]:
from music21 import *
import random
import os

def generate_melody_from_color(r, g, b, length):
    # Map RGB to a musical scale
    scales = ['C', 'G', 'D', 'A', 'E', 'B', 'F#', 'C#']
    scale_index = (r + g + b) % len(scales)
    scale_name = scales[scale_index]

    # Use color values to influence note choices
    sc = scale.MajorScale(scale_name)
    melody = stream.Stream()

    for _ in range(length):
        # Use color values to bias note selection
        pitch_index = (r + g + b) % len(sc.getPitches())
        note_name = sc.getPitches()[pitch_index]
        n = note.Note(note_name)
        n.quarterLength = random.choice([0.5, 1, 1.5, 2])  # Random duration
        melody.append(n)

    return melody

# Example usage
r, g, b = 120, 80, 200  # Example color
color_melody = generate_melody_from_color(r, g, b, 16)

# Save as MIDI
mf = midi.translate.streamToMidiFile(color_melody)
mf.open('color_melody.mid', 'wb')
mf.write()
mf.close()

print("Melody has been generated and saved as 'color_melody.mid'")

# Display a download link for the MIDI file
from google.colab import files
files.download('color_melody.mid')

# Optional: Print out the notes in the melody
print("Notes in the melody:")
for note_obj in color_melody.notes:
    print(f"Note: {note_obj.nameWithOctave}, Duration: {note_obj.quarterLength}")

Melody has been generated and saved as 'color_melody.mid'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Notes in the melody:
Note: C4, Duration: 2.0
Note: C4, Duration: 1.0
Note: C4, Duration: 1.0
Note: C4, Duration: 1.5
Note: C4, Duration: 2.0
Note: C4, Duration: 1.5
Note: C4, Duration: 1.0
Note: C4, Duration: 2.0
Note: C4, Duration: 0.5
Note: C4, Duration: 1.0
Note: C4, Duration: 2.0
Note: C4, Duration: 2.0
Note: C4, Duration: 2.0
Note: C4, Duration: 1.0
Note: C4, Duration: 1.5
Note: C4, Duration: 0.5


In [6]:
!pip install pyo
!pip install rembg


In [7]:
!pip install --upgrade numpy scikit-learn colormath

  Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import MiniBatchKMeans
import io
import logging
import traceback
import time
import random
from rembg import remove
from scipy import signal
import gradio as gr
import numpy as np
from sklearn.cluster import KMeans
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from functools import partial
from colormath.color_diff import delta_e_cie2000 as original_delta_e_cie2000
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color


def rgb_to_lab(rgb):
    r, g, b = rgb[0]/255.0, rgb[1]/255.0, rgb[2]/255.0

    if r > 0.04045:
        r = ((r + 0.055) / 1.055) ** 2.4
    else:
        r = r / 12.92
    if g > 0.04045:
        g = ((g + 0.055) / 1.055) ** 2.4
    else:
        g = g / 12.92
    if b > 0.04045:
        b = ((b + 0.055) / 1.055) ** 2.4
    else:
        b = b / 12.92

    x = (r * 0.4124 + g * 0.3576 + b * 0.1805) / 0.95047
    y = (r * 0.2126 + g * 0.7152 + b * 0.0722) / 1.00000
    z = (r * 0.0193 + g * 0.1192 + b * 0.9505) / 1.08883

    x = x ** (1/3) if x > 0.008856 else (7.787 * x) + 16/116
    y = y ** (1/3) if y > 0.008856 else (7.787 * y) + 16/116
    z = z ** (1/3) if z > 0.008856 else (7.787 * z) + 16/116

    return [(116 * y) - 16, 500 * (x - y), 200 * (y - z)]

def color_difference(lab1, lab2):
    return sum((a - b) ** 2 for a, b in zip(lab1, lab2)) ** 0.5

def advanced_color_analysis(image, n_colors=5):
    # Resize image for faster processing
    image = image.resize((100, 100))

    # Convert image to RGB if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = np.array(image)

    # Reshape the pixels for KMeans
    pixels = pixels.reshape(-1, 3)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_colors, n_init=10)
    kmeans.fit(pixels)

    # Get the dominant colors
    dominant_colors = kmeans.cluster_centers_.astype(int)

    # Calculate color percentages
    color_percentages = np.bincount(kmeans.labels_) / len(kmeans.labels_)

    # Convert dominant colors to Lab color space
    lab_colors = [rgb_to_lab(color) for color in dominant_colors]

    # Calculate color differences
    color_differences = []
    for i in range(len(lab_colors)):
        for j in range(i+1, len(lab_colors)):
            diff = color_difference(lab_colors[i], lab_colors[j])
            color_differences.append(diff)

    # Calculate overall color mood
    average_lab = np.mean([color[0] for color in lab_colors])
    mood = "warm" if average_lab > 50 else "cool"

    # Calculate color variance (as a measure of contrast)
    color_variance = np.var(dominant_colors, axis=0).mean()

    return {
        "dominant_colors": dominant_colors,
        "color_percentages": color_percentages,
        "color_differences": color_differences,
        "mood": mood,
        "contrast": float(color_variance)
    }

def generate_note(frequency, duration, sample_rate=44100, instrument='sine'):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    note = INSTRUMENTS[instrument](frequency, t)

    # Apply a simple envelope
    attack_samples = int(sample_rate * 0.01)  # 10ms attack
    decay_samples = int(sample_rate * 0.01)   # 10ms decay
    total_samples = len(note)

    if total_samples > (attack_samples + decay_samples):
        envelope = np.concatenate([
            np.linspace(0, 1, attack_samples),
            np.ones(total_samples - attack_samples - decay_samples),
            np.linspace(1, 0, decay_samples)
        ])
    else:
        # If the note is very short, use a simpler envelope
        envelope = np.linspace(0, 1, total_samples//2)
        envelope = np.concatenate([envelope, envelope[::-1]])

    return note * envelope

# def advanced_color_analysis(image, n_colors=5):
#     # Resize image for faster processing
#     image = image.resize((100, 100))

#     # Convert image to RGB if it's not already
#     if image.mode != 'RGB':
#         image = image.convert('RGB')

#     pixels = np.array(image)

#     # Reshape the pixels for KMeans
#     pixels = pixels.reshape(-1, 3)

#     # Perform KMeans clustering
#     kmeans = KMeans(n_clusters=n_colors, n_init=10)
#     kmeans.fit(pixels)

#     # Get the dominant colors
#     dominant_colors = kmeans.cluster_centers_.astype(int)

#     # Calculate color percentages
#     color_percentages = np.bincount(kmeans.labels_) / len(kmeans.labels_)

#     # Convert dominant colors to Lab color space
#     lab_colors = [convert_color(sRGBColor(r/255, g/255, b/255), LabColor) for r, g, b in dominant_colors]

#     # Use a partial function to replace numpy.asscalar with our safe_asscalar
#     safe_delta_e_cie2000 = partial(original_delta_e_cie2000, asscalar=safe_asscalar)

#     # Calculate color differences
#     color_differences = []
#     for i in range(len(lab_colors)):
#         for j in range(i+1, len(lab_colors)):
#             diff = safe_delta_e_cie2000(lab_colors[i], lab_colors[j])
#             color_differences.append(diff)

#     # Calculate overall color mood
#     average_lab = np.mean([color.lab_l for color in lab_colors])
#     mood = "warm" if average_lab > 50 else "cool"

#     # Calculate color variance (as a measure of contrast)
#     color_variance = np.var(dominant_colors, axis=0).mean()

#     return {
#         "dominant_colors": dominant_colors,
#         "color_percentages": color_percentages,
#         "color_differences": color_differences,
#         "mood": mood,
#         "contrast": float(color_variance)
#     }

# Define musical scales and instruments
SCALES = {
    'pentatonic': [0, 2, 4, 7, 9],
    'major': [0, 2, 4, 5, 7, 9, 11],
    'minor': [0, 2, 3, 5, 7, 8, 10]
}

INSTRUMENTS = {
    'sine': lambda freq, t: np.sin(2 * np.pi * freq * t),
    'square': lambda freq, t: signal.square(2 * np.pi * freq * t),
    'sawtooth': lambda freq, t: signal.sawtooth(2 * np.pi * freq * t),
    'triangle': lambda freq, t: signal.sawtooth(2 * np.pi * freq * t, width=0.5),
    'piano': lambda freq, t: np.sin(2 * np.pi * freq * t) * np.exp(-0.0004 * 2 * np.pi * freq * t),
    'guitar': lambda freq, t: signal.sawtooth(2 * np.pi * freq * t) * np.exp(-0.0015 * 2 * np.pi * freq * t),
    'flute': lambda freq, t: (np.sin(2 * np.pi * freq * t) + 0.1 * np.sin(4 * np.pi * freq * t)) * np.exp(-0.0004 * 2 * np.pi * freq * t)
}

RHYTHM_PATTERNS = {
    'simple': [1, 0, 1, 0],
    'syncopated': [1, 0, 0.5, 0, 1, 0.5],
    'waltz': [1, 0.5, 0.5, 1, 0.5, 0.5],
    'complex': [1, 0, 0.5, 0.25, 0.75, 0, 0.5, 0.25]
}

def get_rhythm_pattern(pattern_name, beats):
    patterns = {
        'simple': [1, 0, 1, 0],
        'syncopated': [1, 0, 0.5, 0, 1, 0.5],
        'waltz': [1, 0.5, 0.5, 1, 0.5, 0.5],
        'complex': [1, 0, 0.5, 0.25, 0.75, 0, 0.5, 0.25]
    }
    pattern = patterns.get(pattern_name, patterns['simple'])
    return (pattern * (beats // len(pattern) + 1))[:beats]

def color_to_pitch(color, scale):
    # Map color to a pitch in the given scale
    hue = (color[0] + color[1] + color[2]) / 3  # Simple hue calculation
    pitch_index = int(hue / 255 * len(scale))
    return scale[pitch_index]

def color_to_duration(percentage, base_duration):
    # Map color percentage to note duration
    return base_duration * (1 + percentage)

def contrast_to_tempo(contrast, min_tempo=60, max_tempo=180):
    # Map contrast to tempo
    return min_tempo + (contrast / 255) * (max_tempo - min_tempo)

def mood_to_scale(mood):
    # Define scales based on mood
    if mood == "warm":
        return [0, 2, 4, 5, 7, 9, 11]  # Major scale
    else:
        return [0, 2, 3, 5, 7, 8, 10]  # Natural minor scale

def color_diff_to_harmony(diff, threshold=30):
    # Determine if two colors should be harmonically related
    return diff < threshold

def generate_chord(base_freq, scale, chord_type='triad'):
    if chord_type == 'triad':
        chord_steps = [0, 2, 4]
    elif chord_type == 'seventh':
        chord_steps = [0, 2, 4, 6]
    else:
        raise ValueError("Unsupported chord type")

    return [base_freq * (2 ** (scale[step % len(scale)] / 12)) for step in chord_steps]

def get_dominant_colors(image, n_colors=8):
    # Resize image for faster processing, but not too small
    image = image.resize((200, 200))

    # Convert image to RGB if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = np.array(image)
    pixels = pixels.reshape(-1, 3)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_colors, n_init=10, random_state=42)
    kmeans.fit(pixels)

    # Get the colors and their percentages
    colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_
    percentages = np.bincount(labels) / len(labels)

    # Sort colors by percentage
    sorted_indices = np.argsort(percentages)[::-1]
    colors = colors[sorted_indices]
    percentages = percentages[sorted_indices]

    return colors, percentages

def improved_color_to_frequency(r, g, b, base_freq=220, scale='pentatonic'):
    # Convert RGB to HSL
    r, g, b = r/255.0, g/255.0, b/255.0
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    l = (cmax + cmin) / 2

    if cmax == cmin:
        h = 0
    elif cmax == r:
        h = (g - b) / (cmax - cmin)
    elif cmax == g:
        h = 2 + (b - r) / (cmax - cmin)
    else:
        h = 4 + (r - g) / (cmax - cmin)

    h = (h * 60) % 360

    # Map hue to scale degree
    scale_degree = int((h / 360) * len(SCALES[scale]))

    # Calculate frequency
    freq = base_freq * (2 ** (SCALES[scale][scale_degree] / 12))

    # Use lightness to adjust octave
    octave_adjust = int(l * 3) - 1  # -1, 0, or 1
    freq *= (2 ** octave_adjust)

    return freq

def generate_note(frequency, duration, sample_rate=44100, instrument='sine'):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    note = INSTRUMENTS[instrument](frequency, t)

    # Apply a simple envelope
    envelope = np.linspace(0, 1, int(sample_rate * 0.01))
    envelope = np.concatenate([envelope, np.ones(len(note) - 2*len(envelope)), envelope[::-1]])
    return note * envelope

def create_advanced_audio(color_analysis, duration, tempo, scale, instrument, is_background, rhythm_pattern='simple'):
    try:
        sample_rate = 44100
        base_freq = 220 if is_background else 440  # A3 for background, A4 for foreground

        beat_duration = max(60 / tempo, 0.1)  # Ensure minimum beat duration of 0.1 seconds
        total_beats = int(duration / beat_duration)
        audio = np.zeros(int(sample_rate * duration))

        rhythm = get_rhythm_pattern(rhythm_pattern, total_beats)

        colors = color_analysis["dominant_colors"]
        percentages = color_analysis["color_percentages"]

        # Normalize percentages to ensure they sum to 1
        percentages = percentages / np.sum(percentages)

        beat = 0
        while beat < total_beats:
            for color, percentage in zip(colors, percentages):
                color_beats = max(1, int(percentage * total_beats))

                for _ in range(color_beats):
                    if beat >= total_beats:
                        break

                    pitch = color_to_pitch(color, scale)
                    freq = base_freq * (2 ** (pitch / 12))

                    if is_background:
                        # For background, create sustained chords
                        chord_freqs = generate_chord(freq, scale)
                        note = np.sum([generate_note(f, beat_duration * 2, sample_rate, instrument) for f in chord_freqs], axis=0)
                        note = note / len(chord_freqs)  # Normalize volume
                    else:
                        # For foreground, create a melody
                        note_duration = beat_duration * (1 + random.random())  # Vary note length
                        note = generate_note(freq, note_duration, sample_rate, instrument)

                        # Add some variation
                        if random.random() > 0.7:
                            # Occasionally add a second note
                            second_freq = base_freq * (2 ** ((pitch + 2) / 12))  # Add a third
                            second_note = generate_note(second_freq, note_duration, sample_rate, instrument)
                            note = (note + second_note) / 2

                    # Apply rhythm
                    note *= rhythm[beat % len(rhythm)]

                    # Add note to audio
                    start = int(beat * beat_duration * sample_rate)
                    end = start + len(note)
                    if end > len(audio):
                        audio[start:] += note[:len(audio)-start]
                    else:
                        audio[start:end] += note

                    beat += 1
                    if beat >= total_beats:
                        break

        # Apply effects
        if is_background:
            b, a = signal.butter(10, 0.5)
            audio = signal.filtfilt(b, a, audio)
        else:
            # Add a slight echo effect to the foreground
            echo = np.zeros_like(audio)
            echo[int(0.1 * sample_rate):] = audio[:-int(0.1 * sample_rate)] * 0.3
            audio += echo

        # Normalize audio
        audio = audio / np.max(np.abs(audio))

        return audio, sample_rate

    except Exception as e:
        logging.error(f"Error in create_advanced_audio: {str(e)}")
        logging.error(traceback.format_exc())
        raise

def create_color_info(fg_color_analysis, bg_color_analysis, overall_mood, overall_contrast):
    # Create color swatches
    fg_swatches = Image.new('RGB', (400, 50))
    bg_swatches = Image.new('RGB', (400, 50))

    for i, color in enumerate(fg_color_analysis["dominant_colors"]):
        fg_swatches.paste(Image.new('RGB', (50, 50), tuple(color)), (i*50, 0))

    for i, color in enumerate(bg_color_analysis["dominant_colors"]):
        bg_swatches.paste(Image.new('RGB', (50, 50), tuple(color)), (i*50, 0))

    combined_swatches = Image.new('RGB', (400, 100))
    combined_swatches.paste(fg_swatches, (0, 0))
    combined_swatches.paste(bg_swatches, (0, 50))

    # Create color information text
    color_text = "Foreground Colors:\n"
    for color, percentage in zip(fg_color_analysis["dominant_colors"], fg_color_analysis["color_percentages"]):
        color_text += f"RGB{tuple(color)}: {percentage:.2%}\n"

    color_text += "\nBackground Colors:\n"
    for color, percentage in zip(bg_color_analysis["dominant_colors"], bg_color_analysis["color_percentages"]):
        color_text += f"RGB{tuple(color)}: {percentage:.2%}\n"

    color_text += f"\nOverall Mood: {overall_mood.capitalize()}"

    color_text += f"\nContrast: {overall_contrast:.2f}"

    return combined_swatches, color_text

def separate_foreground_background(image):
    # Convert image to RGBA if it's not already
    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # Remove background
    foreground = remove(np.array(image))

    # Create a mask of the foreground
    mask = foreground[:,:,3] > 0

    # Create background by masking out the foreground
    background = np.array(image)
    background[mask] = [255, 255, 255, 255]  # Set to white

    # Convert foreground and background to RGB
    foreground_rgb = Image.fromarray(foreground).convert('RGB')
    background_rgb = Image.fromarray(background).convert('RGB')

    return foreground_rgb, background_rgb

def save_audio_file(audio_data, sample_rate, filename):
    from scipy.io import wavfile
    wavfile.write(filename, sample_rate, audio_data)
    return filename

def rgb_to_lab(rgb):
    r, g, b = rgb
    r, g, b = r/255.0, g/255.0, b/255.0

    def f(t):
        return t**(1/3) if t > 0.008856 else 7.787*t + 16/116

    x = f(0.412453*r + 0.357580*g + 0.180423*b)
    y = f(0.212671*r + 0.715160*g + 0.072169*b)
    z = f(0.019334*r + 0.119193*g + 0.950227*b)

    L = 116 * y - 16
    a = 500 * (x - y)
    b = 200 * (y - z)

    return [L, a, b]

def advanced_process_image(image, duration, bpm, scale='major', fg_instrument='sine', bg_instrument='piano', rhythm_pattern='simple'):
    try:
        start_time = time.time()
        logging.info("Starting advanced image processing")

        # Separate foreground and background
        foreground, background = separate_foreground_background(image)

        # Extract colors
        fg_colors, fg_percentages = get_dominant_colors(foreground)
        bg_colors, bg_percentages = get_dominant_colors(background)

        # Create color analysis dictionaries
        fg_color_analysis = {
            "dominant_colors": fg_colors,
            "color_percentages": fg_percentages
        }
        bg_color_analysis = {
            "dominant_colors": bg_colors,
            "color_percentages": bg_percentages
        }

        # Determine overall mood and contrast
        fg_lab_colors = [rgb_to_lab(color) for color in fg_colors]
        bg_lab_colors = [rgb_to_lab(color) for color in bg_colors]
        fg_average_l = np.mean([lab[0] for lab in fg_lab_colors])
        bg_average_l = np.mean([lab[0] for lab in bg_lab_colors])
        overall_mood = "warm" if (fg_average_l + bg_average_l) / 2 > 50 else "cool"
        overall_contrast = np.std(fg_lab_colors + bg_lab_colors, axis=0)[0]

        # Set musical parameters
        if scale == 'auto':
            scale = mood_to_scale(overall_mood)
        else:
            scale = SCALES[scale]
        tempo = contrast_to_tempo(overall_contrast) if bpm == 0 else bpm

        # Generate audio
        fg_audio_data, fg_sample_rate = create_advanced_audio(fg_color_analysis, duration, tempo, scale, fg_instrument, is_background=False, rhythm_pattern=rhythm_pattern)
        bg_audio_data, bg_sample_rate = create_advanced_audio(bg_color_analysis, duration, tempo, scale, bg_instrument, is_background=True, rhythm_pattern=rhythm_pattern)

        # Mix foreground and background audio
        mixed_audio = fg_audio_data * 0.6 + bg_audio_data * 0.4  # Adjust mix ratio
        mixed_audio = np.int16(mixed_audio / np.max(np.abs(mixed_audio)) * 32767)

        # Create color swatches and information text
        combined_swatches, color_text = create_color_info(fg_color_analysis, bg_color_analysis, overall_mood, overall_contrast)

        # Save audio file
        audio_file = save_audio_file(mixed_audio, fg_sample_rate, "palette_music.wav")

        logging.info(f"Advanced image processing completed in {time.time() - start_time:.2f} seconds")
        return combined_swatches, color_text, (fg_sample_rate, mixed_audio), audio_file

    except Exception as e:
        logging.error(f"Error in advanced_process_image: {str(e)}")
        logging.error(traceback.format_exc())
        return None, f"An error occurred: {str(e)}", None, None

# Define Gradio interface
iface = gr.Interface(
    fn=advanced_process_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Slider(minimum=5, maximum=30, value=10, step=1, label="Duration (seconds)"),
        gr.Slider(minimum=0, maximum=180, value=0, step=1, label="Tempo (BPM, 0 for auto)"),
        gr.Dropdown(choices=['auto'] + list(SCALES.keys()), label="Musical Scale", value='auto'),
        gr.Dropdown(choices=list(INSTRUMENTS.keys()), label="Foreground Instrument", value="sine"),
        gr.Dropdown(choices=list(INSTRUMENTS.keys()), label="Background Instrument", value="piano"),
        gr.Dropdown(choices=list(RHYTHM_PATTERNS.keys()), label="Rhythm Pattern", value="simple")
    ],
    outputs=[
        gr.Image(type="pil", label="Dominant Colors (Foreground top, Background bottom)"),
        gr.Textbox(label="Color Information"),
        gr.Audio(type="numpy", label="Palette Music"),
        gr.File(label="Download Audio")
    ],
    title="Advanced Image Color Palette to Music Converter",
    description="Upload an image to extract its dominant colors and hear them as a sophisticated musical composition!"
)

# Launch the interface
if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5c4d1ba1975124a903.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
